In [2]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime
import dateutil.parser



In [70]:

#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='+str(sub)+'&size=100&after='+str(after)+'&before='+str(before)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']


In [83]:
url='https://api.pushshift.io/reddit/search/submission/?subreddit=DadJokes&size=100&after=1489541943&before=1604034000'
r = requests.get(url)
data = json.loads(r.text)

In [71]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    try:
        punchline = subm['selftext']
    except KeyError:
        punchline = "NaN"
    url = subm['url']
    #flairs are not always present so we wrap in try/except    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,punchline,url,author,score,created,numComms,permalink))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

In [103]:
subCount = 0
subStats = {}
after = '18/10/2016'
after = int(dateutil.parser.parse(after, dayfirst=True).timestamp())
print(after)
before= '30/10/2020'
before = int(dateutil.parser.parse(before, dayfirst=True).timestamp())
print(before)

1476766800
1604034000


In [107]:
data = getPushshiftData(after, before, sub)
# We need to run this function outside the loop first to get the updated after variable
#data = getPushshiftData(after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    print(after)
    #data has changed due to the new after variable provided by above code
    try:
        data = getPushshiftData(after, before, sub)
    except:
        after = data[-2]['created_utc']

    

https://api.pushshift.io/reddit/search/submission/?subreddit=DadJokes&size=100&after=1480346787&before=1604034000


In [109]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

142294 submissions have added to list
1st entry is:
I texted this to my girlfriend yesterday after she fell asleep on me created: 3
Last entry is:
I was waiting to meet a friend on a cold and icy day. created: 1


In [111]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Punchline","Url","Author","Score","Publish Date","Total No. of Comments","Permalink"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

input filename of submission file, please add .csv


 dadjokes.csv


142294 submissions have been uploaded


NameError: name 'subm' is not defined